In [2]:
import polars as pl
from python_scripts import *
from config import *
import duckdb

In [3]:
df1 = pl.read_parquet('jobs_from_7.0_days_2025-04-18.parquet').rename(
    {"Seniority level": "seniority_level",
     "Employment type": "employment_type",
     "Job function": "job_function",
     "Industries": "industries",}
)
df2 = pl.read_csv('joblist_2025-04-13_.csv').drop('cloud_focus')

In [4]:
# --- 1. Configuration ---
db_file = 'my_project.duckdb' # Use .duckdb extension by convention
table_name = 'test_table' #Choose a name for your SQL table


In [5]:
print(df1.count())

shape: (1, 10)
┌───────┬─────────┬──────────┬──────┬───┬───────────────┬──────────────┬──────────────┬────────────┐
│ title ┆ company ┆ location ┆ date ┆ … ┆ seniority_lev ┆ employment_t ┆ job_function ┆ industries │
│ ---   ┆ ---     ┆ ---      ┆ ---  ┆   ┆ el            ┆ ype          ┆ ---          ┆ ---        │
│ u32   ┆ u32     ┆ u32      ┆ u32  ┆   ┆ ---           ┆ ---          ┆ u32          ┆ u32        │
│       ┆         ┆          ┆      ┆   ┆ u32           ┆ u32          ┆              ┆            │
╞═══════╪═════════╪══════════╪══════╪═══╪═══════════════╪══════════════╪══════════════╪════════════╡
│ 31    ┆ 31      ┆ 31       ┆ 31   ┆ … ┆ 31            ┆ 31           ┆ 31           ┆ 31         │
└───────┴─────────┴──────────┴──────┴───┴───────────────┴──────────────┴──────────────┴────────────┘


In [6]:
con = duckdb.connect("my_project.duckdb")

In [7]:
con.sql("""SELECT COUNT(*) FROM test_table""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│           65 │
└──────────────┘

In [8]:
con.sql("""
        SELECT *
        FROM test_table
        """)

┌────────────────────────────────────────────────┬─────────────────────┬───────────────────────────────────────────────┬────────────┬────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [ ]:
df_from_db = con.execute("""SELECT * 
                         FROM test_table t1

                         WHERE job_url NOT IN (SELECT job_url FROM genai_table)

                         """).fetchdf()
df_from_db = pl.DataFrame(df_from_db)
print(df_from_db)

In [ ]:
genai_list: list[dict] = []
for row in df_from_db.rows(named=True):
    
    genai_data: dict = {}
    genai_data.update(classify_job_description(row['job_description']))
    genai_data.update({'job_url': row['job_url']})
    genai_list.append(genai_data)
    
print(genai_list)

In [12]:
df_genai = pl.DataFrame(genai_list)

In [13]:
insert_into_db(df_genai, "genai_table", "my_project.duckdb", "genai_data")


--- Performing UPSERT for table 'genai_table' on key 'job_url' ---
Registered DataFrame as view 'df_upsert_view'
Executed: CREATE TABLE IF NOT EXISTS genai_table(
                                    job_url VARCHAR PRIMARY KEY,
                                    task_clarity VARCHAR,
                                    seniority_level_ai VARCHAR,
                                    requires_degree_it VARCHAR,
                                    mentions_certifications VARCHAR,
                                    years_of_experience VARCHAR,
                                    is_in_english VARCHAR,
                                    cloud_preference VARCHAR,
                                    skills_mentioned VARCHAR[]
                                    );
                                     (if table didn't exist)

Executing UPSERT SQL:

        INSERT INTO genai_table ("task_clarity", "seniority_level_ai", "requires_degree_it", "mentions_certifications", "years_of_experience", 

In [19]:
df_results = pl.DataFrame(results)

In [23]:
con.sql("""
        SELECT t1.*, t2.*
        FROM test_table t1

        JOIN genai_table t2
        ON t1.job_url = t2.job_url

        WHERE t1.job_url = 'https://www.linkedin.com/jobs/view/4207190424/'
        """)

┌──────────────────────┬──────────────┬───────────────────────────────────────────────┬────────────┬────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [10]:
con = duckdb.connect("my_project.duckdb")

In [11]:
con.sql("""
        SELECT t1.*, t2.*
        FROM test_table t1
        JOIN genai_table t2
        ON t1.job_url = t2.job_url
        QUALIFY ROW_NUMBER() OVER (PARTITION BY t1.title, t1.company, t1.date ORDER BY t1.date DESC) = 1
        ORDER BY t1.title, t1.company, t1.date
        """).fetchdf()

,title,company,location,date,job_url,job_description,seniority_level,employment_type,job_function,industries,job_url_1,task_clarity,seniority_level_ai,requires_degree_it,mentions_certifications,years_of_experience,is_in_english,cloud_preference,skills_mentioned
0,Analista Data Engineer AzureDataBricks,Accenture Chile,"Santiago, Santiago Metropolitan Region, Chile",2025-05-13,https://www.linkedin.com/jobs/view/4212693987/,WORK AT THE HEART OF THE CHANGE\nEmprende una ...,entry_level,full-time,information_technology,business_consulting_and_services,https://www.linkedin.com/jobs/view/4212693987/,Medium,Junior,No,Yes,1,No,Azure,"[Databricks or snowflake, Develop pipelines or..."
1,Chief Data Engineer,EPAM Systems,Chile,2025-04-17,https://www.linkedin.com/jobs/view/4212167524/,We are seeking an experienced Chief Data Engin...,Mid-Senior level,Full-time,"Engineering, Information Technology, and Busin...","Software Development, IT Services and IT Consu...",https://www.linkedin.com/jobs/view/4212167524/,High,Lead or greater,Yes,No,7+,Yes,AWS,"[Databricks or snowflake, Develop pipelines or..."
2,Chief Data Engineer,EPAM Systems,Chile,2025-05-01,https://www.linkedin.com/jobs/view/4220611480/,We are seeking an experienced \nChief Data Eng...,mid-senior_level,full-time,"engineering,_information_technology,_and_busin...","software_development,_it_services_and_it_consu...",https://www.linkedin.com/jobs/view/4220611480/,High,Lead or greater,Yes,No,7+,Yes,AWS,"[Develop pipelines or ETL/ELT processes, Data ..."
3,Consultor Data Engineer,MAS Analytics,"Santiago, Santiago Metropolitan Region, Chile",2025-05-07,https://www.linkedin.com/jobs/view/4223641021/,MAS Analytics\n es una consultora de datos e i...,not_applicable,full-time,information_technology,"technology,_information_and_internet_and_infor...",https://www.linkedin.com/jobs/view/4223641021/,Medium,Junior,Yes,No,Not Specified,No,Multiple Clouds,"[Develop pipelines or ETL/ELT processes, Data ..."
4,Data Engineer,2Brains,Chile,2025-04-10,https://www.linkedin.com/jobs/view/4205294882/,2Brains es una empresa dedicada a construir y ...,Mid-Senior level,Full-time,Information Technology,"Technology, Information and Internet and Infor...",https://www.linkedin.com/jobs/view/4205294882/,High,Mid-Senior,No,No,Not Specified,No,AWS,"[Develop pipelines or ETL/ELT processes, Data ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,Senior Data Engineer,Ottomatik.io,"Santiago, Santiago Metropolitan Region, Chile",2025-04-12,https://www.linkedin.com/jobs/view/4207190424/,Hi there! We are South and our client is looki...,Executive,Full-time,"Business Development, Consulting, and Engineering",IT System Data Services,https://www.linkedin.com/jobs/view/4207190424/,Medium,Senior,Yes,No,4,Yes,AWS,"[Develop pipelines or ETL/ELT processes, Data ..."
106,Senior Data Engineer,Sparq,Chile,2025-04-15,https://www.linkedin.com/jobs/view/4210998431/,Team Sparq is committed to creating high-quali...,Mid-Senior level,Full-time,Information Technology,Software Development,https://www.linkedin.com/jobs/view/4210998431/,High,Senior,No,No,5,Yes,Azure,"[Databricks or snowflake, Develop pipelines or..."
107,Senior Data Engineer Python,23people,Chile,2025-04-10,https://www.linkedin.com/jobs/view/4205294906/,Únete a Equifax Chile como Senior Data Enginee...,Mid-Senior level,Full-time,Information Technology,"Technology, Information and Internet and Infor...",https://www.linkedin.com/jobs/view/4205294906/,Medium,Senior,No,No,5,No,GCP,"[Develop pipelines or ETL/ELT processes, Versi..."
108,"Sr. Data Engineer (@Remote, Chile and Colombia)",OfferUp,Chile,2025-05-15,https://www.linkedin.com/jobs/view/4230935527/,About The Role\nWe are looking for an experien...,mid-senior_level,full-time,information_technology,"technology,_information_and_internet",https://www.linkedin.com/jobs/view/4230935527/,High,Lead or greater,Yes,No,5,Yes,AWS,"[Develop pipelines or ETL/ELT processes, Spark..."


In [5]:
con.sql("""
ATTACH 'output/my_project.duckdb' AS output_db;
        """)

In [8]:
con2.sql("""
        SELECT t1.*, t2.*
        FROM base_table t1
        JOIN genai_table t2
        ON t1.job_url = t2.job_url
        QUALIFY ROW_NUMBER() OVER (PARTITION BY t1.title, t1.company, t1.date ORDER BY t1.date DESC) = 1
        ORDER BY t1.title, t1.company, t1.date
        """).fetchdf()

,title,company,location,date,job_url,job_description,seniority_level,employment_type,job_function,industries,job_url_1,task_clarity,seniority_level_ai,requires_degree_it,mentions_certifications,years_of_experience,is_in_english,cloud_preference,skills_mentioned
0,Chief Data Engineer,EPAM Systems,Chile,2025-05-01,https://www.linkedin.com/jobs/view/4220611480/,We are seeking an experienced \nChief Data Eng...,mid-senior_level,full-time,"engineering,_information_technology,_and_busin...","software_development,_it_services_and_it_consu...",https://www.linkedin.com/jobs/view/4220611480/,High,Lead or greater,Yes,No,7+,Yes,AWS,"[Develop pipelines or ETL/ELT processes, Data ..."
1,Chief Data Engineer,EPAM Systems,Chile,2025-05-09,https://www.linkedin.com/jobs/view/4212167524/,We are seeking an experienced Chief Data Engin...,mid-senior_level,full-time,"engineering,_information_technology,_and_busin...","software_development,_it_services_and_it_consu...",https://www.linkedin.com/jobs/view/4212167524/,High,Lead or greater,Yes,No,7+,Yes,AWS,"[Develop pipelines or ETL/ELT processes, Data ..."
2,Consultor Data Engineer,MAS Analytics,"Santiago, Santiago Metropolitan Region, Chile",2025-05-07,https://www.linkedin.com/jobs/view/4223641021/,MAS Analytics\n es una consultora de datos e i...,not_applicable,full-time,information_technology,"technology,_information_and_internet_and_infor...",https://www.linkedin.com/jobs/view/4223641021/,High,Junior,Yes,No,Not Specified,No,Multiple Clouds,"[Develop pipelines or ETL/ELT processes, Data ..."
3,Data Engineer,2Brains,Chile,2025-04-29,https://www.linkedin.com/jobs/view/4216874808/,2Brains es una empresa dedicada a construir y ...,mid-senior_level,full-time,information_technology,"technology,_information_and_internet_and_infor...",https://www.linkedin.com/jobs/view/4216874808/,High,Senior,No,Yes,5+,No,Multiple Clouds,"[Develop pipelines or ETL/ELT processes, Data ..."
4,Data Engineer,ARKHO,Chile,2025-05-07,https://www.linkedin.com/jobs/view/4223640231/,ARKHO es una consultora experta en tecnologías...,mid-senior_level,full-time,information_technology,"technology,_information_and_internet_and_infor...",https://www.linkedin.com/jobs/view/4223640231/,Medium,Mid-Senior,No,Yes,Not Specified,No,AWS,"[Develop pipelines or ETL/ELT processes, Data ..."
5,Data Engineer,EPAM Systems,Chile,2025-05-07,https://www.linkedin.com/jobs/view/4225594752/,We are seeking a highly skilled and motivated ...,associate,full-time,"information_technology,_engineering,_and_busin...","software_development,_it_services_and_it_consu...",https://www.linkedin.com/jobs/view/4225594752/,High,Mid-Senior,No,No,2+,Yes,Azure,"[Develop pipelines or ETL/ELT processes, Spark..."
6,Data Engineer,Falabella,"Santiago, Santiago Metropolitan Region, Chile",2025-05-02,https://www.linkedin.com/jobs/view/4207045620/,Descripción Empresa\nSomos más de 80 mil perso...,mid-senior_level,full-time,information_technology,retail,https://www.linkedin.com/jobs/view/4207045620/,High,Mid-Senior,Yes,No,Not Specified,No,GCP,"[Develop pipelines or ETL/ELT processes, Data ..."
7,Data Engineer,Falabella,"Santiago, Santiago Metropolitan Region, Chile",2025-05-07,https://www.linkedin.com/jobs/view/4225586690/,Descripción Empresa\nSomos más de 80 mil perso...,mid-senior_level,full-time,information_technology,retail,https://www.linkedin.com/jobs/view/4225586690/,Medium,Mid-Senior,Yes,No,Not Specified,No,GCP,"[Develop pipelines or ETL/ELT processes, Data ..."
8,Data Engineer,Genesys,"Santiago, Santiago Metropolitan Region, Chile",2025-04-30,https://www.linkedin.com/jobs/view/4218817757/,Somos una empresa con más de 34 años de experi...,mid-senior_level,full-time,information_technology,software_development_and_financial_services,https://www.linkedin.com/jobs/view/4218817757/,High,Mid-Senior,No,No,3+,No,AWS,"[Develop pipelines or ETL/ELT processes, Data ..."
9,Data Engineer,NeuralWorks,"Santiago, Santiago Metropolitan Region, Chile",2025-04-29,https://www.linkedin.com/jobs/vie

In [6]:
con.sql("""INSERT INTO test_table 
        SELECT * FROM output_db.base_table
        WHERE job_url NOT IN (SELECT job_url FROM test_table)""")

In [7]:
con.sql("""INSERT INTO genai_table 
        SELECT * FROM output_db.genai_table
        WHERE job_url NOT IN (SELECT job_url FROM genai_table)""")

In [8]:
con.sql("""SELECT COUNT(*) FROM genai_table""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│          131 │
└──────────────┘

In [9]:
con.close()